In [33]:
import json
import duckdb
import torch
import numpy as np
from pathlib import Path
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from tqdm.auto import tqdm

In [10]:
%%sql

select * from submissions


In [7]:
%%sql
select * from submissions where selftext like '[removed]'

,id,author,created_at,subreddit,title,selftext,score,upvote_ratio,num_comments,url,domain
0,1l0bg38,AdamNW,2025-06-01T00:05Z,AskConservatives,What exactly is meant when someone says they d...,[removed],1,1.0,1,https://www.reddit.com/r/AskConservatives/comm...,self.AskConservatives
1,1l0bgcx,Ana-lorde748,2025-06-01T00:05:24Z,unitedkingdom,If you love me Message me,[removed],1,1.0,0,https://www.reddit.com/r/unitedkingdom/comment...,self.unitedkingdom
2,1l0bkzz,Andyqpr_YT,2025-06-01T00:12:05Z,unitedkingdom,Fake id,[removed],1,1.0,2,https://www.reddit.com/r/unitedkingdom/comment...,self.unitedkingdom
3,1l0bmnn,[deleted],2025-06-01T00:14:31Z,anime_titties,New Aid Group in Gaza Makes an End Run Around ...,[removed],1,1.0,1,NaN,NaN
4,1l0bno6,[deleted],2025-06-01T00:15:56Z,geopolitics,New Aid Group in Gaza Makes an End Run Around ...,[removed],1,1.0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
22682,1n590ky,Qu3st1onEverything,2025-08-31T23:13:40Z,unitedkingdom,Have the left & right ever spoken to each othe...,[removed],1,1.0,0,https://www.reddit.com/r/unitedkingdom/comment...,self.unitedkingdom
22683,1n597fj,Past_Sandwich1775,2025-08-31T23:22:26Z,conspiracy,Ok kind of a conspiracy,[removed],1,1.0,0,https://www.reddit.com/r/conspiracy/comments/1...,self.conspiracy
22684,1n59e9x,Ok-Lie-9040,2025-08-31T23:31:23Z,Polska,Czy ktoś się wgl cieszy że mieszka w Polsce?,[removed],1,1.0,1,https://www.reddit.com/r/Polska/comments/1n59e...,self.Polska
22685,1n59ftv,True_Ad793,2025-08-31T23:33:21Z,unitedkingdom,Homelessness in England by Region (2005–2016) ...,[removed],1,1.0,0,https://www.youtube.com/watch?v=pGDsrvdWikY,youtube.com


In [17]:
%%sql
DROP TABLE IF EXISTS submissions_filtered;

CREATE TABLE submissions_filtered AS
SELECT *
FROM submissions
WHERE TRIM(selftext) NOT IN ('[removed]', '[deleted]') OR selftext IS NULL;

SELECT * FROM submissions_filtered;

,id,author,created_at,subreddit,title,selftext,score,upvote_ratio,num_comments,url,domain
0,1l0bdii,SmokingBlackSeaFleet,2025-06-01T00:01:27Z,CombatFootage,"Ukraine's ""Da Vinci Wolves"" repelling Russian ...",NaN,399,0.98,5,https://v.redd.it/xzu2pdg8h74f1,/r/CombatFootage/comments/1l0bdii/ukraines_da_...
1,1l0bhmh,MonsterNegra,2025-06-01T00:07:14Z,europe,"Hello everyone, I would like to share the link...",NaN,1,1.00,0,https://rare-world-coins-wsy5bq4.gamma.site/,rare-world-coins-wsy5bq4.gamma.site
2,1l0bhtb,Haunting_Switch3463,2025-06-01T00:07:29Z,europe,'Europe must not shy away from four harsh trut...,NaN,12,0.93,0,https://www.lemonde.fr/en/opinion/article/2025...,lemonde.fr
3,1l0bj65,Puzzled-Category-954,2025-06-01T00:09:24Z,europe,French President posts video thanking Vietnam,NaN,6,0.72,0,https://streamain.com/en/6B9V9AJmGfRxCYU/watch,streamain.com
4,1l0blpy,Enakronizum,2025-06-01T00:13:10Z,conspiracy,Please help me find answers. I am having confl...,NaN,1,1.00,1,https://www.reddit.com/r/conspiracy/comments/1...,self.conspiracy
...,...,...,...,...,...,...,...,...,...,...,...
101442,1n59k9c,True_Ad793,2025-08-31T23:39:18Z,unitedkingdom,UK Gov Data: Homelessness Trends by Region (20...,NaN,1,1.00,0,https://www.youtube.com/watch?v=pGDsrvdWikY,youtube.com
101443,1n59nkb,Mossaaaab,2025-08-31T23:43:41Z,germany,Looking for an internship in Germany in Artifi...,I’m moroccan and currently i’m a last year stu...,0,0.19,8,https://www.reddit.com/r/germany/comments/1n59...,self.germany
101444,1n59pah,callsonreddit,2025-08-31T23:46:02Z,worldnews,US reportedly suspends visa approvals for near...,NaN,6159,0.96,223,https://www.theguardian.com/us-news/2025/aug/3...,theguardian.com
101445,1n59wln,mathew_of_lordran,2025-08-31T23:56Z,worldnews,Video: Videos Contradict Israel’s Rationale fo...,NaN,1,1.00,0,https://www.nytimes.com/video/world/middleeast...,nytimes.com


## Embedding

We use paraphrase-multilingual-MiniLM-L12-v2 from Hugging Face for map sentences & paragraphs to a 384 dimensional dense vector space to filter pro-Ukrainian messages



In [13]:

# Configuration
DB_PATH = Path("/Users/ivantyshchenko/Projects/Python/DataPatron/data/reddit.db")
BATCH_SIZE = 256
device = 'mps' if torch.mps.is_available() else 'cpu'

# Load model
model = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    device=device
)


In [26]:

# Connect to DuckDB
# IMPORTANT: Ensure PyCharm's Database tool is DISCONNECTED from reddit.db before running this.
try:
    conn = duckdb.connect(str(DB_PATH))

    # 1. Create column if needed
    try:
        conn.execute("ALTER TABLE submissions_filtered ADD COLUMN embedding VARCHAR")
    except duckdb.CatalogException:
        pass

    # 2. Fetch data needed for processing
    print("Fetching pending records...")
    pending_records = conn.execute(
        """
        SELECT id, title, selftext
        FROM submissions_filtered
        WHERE embedding IS NULL
          AND (selftext IS NOT NULL OR title IS NOT NULL)
        """
    ).fetchall()

    print(f"Found {len(pending_records)} records to process.")

    # 3. Process and Update
    total_processed = 0

    # Iterate through the fetched list
    for i in range(0, len(pending_records), BATCH_SIZE):
        batch = pending_records[i : i + BATCH_SIZE]

        batch_ids = []
        texts = []

        for row in batch:
            record_id = row[0]
            title = row[1] if row[1] else ""
            selftext = row[2] if row[2] else ""

            full_text = f"{title} {selftext}".strip()

            if len(full_text) > 20 and full_text.lower() not in ('[removed]', '[deleted]'):
                batch_ids.append(record_id)
                texts.append(full_text)

        if not texts:
            continue

        # Generate embeddings
        embeddings = model.encode(texts, convert_to_numpy=True, show_progress_bar=False)

        # Prepare payload
        payload = [
            (json.dumps(vec.tolist()), record_id)
            for vec, record_id in zip(embeddings, batch_ids)
        ]

        # Write to DB
        conn.executemany(
            "UPDATE submissions_filtered SET embedding = ? WHERE id = ?",
            payload
        )

        total_processed += len(batch)
        if total_processed % (BATCH_SIZE * 5) == 0:
            print(f"Processed {total_processed} rows...")

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Crucial: Always close the connection to release the lock file
    if 'conn' in locals():
        conn.close()
        print("Database connection closed.")

Fetching pending records...
Found 101447 records to process.
Processed 1280 rows...
Processed 2560 rows...
Processed 3840 rows...
Processed 5120 rows...
Processed 6400 rows...
Processed 7680 rows...
Processed 8960 rows...
Processed 10240 rows...
Processed 11520 rows...
Processed 12800 rows...
Processed 14080 rows...
Processed 15360 rows...
Processed 16640 rows...
Processed 17920 rows...
Processed 19200 rows...
Processed 20480 rows...
Processed 21760 rows...
Processed 23040 rows...
Processed 24320 rows...
Processed 25600 rows...
Processed 26880 rows...
Processed 28160 rows...
Processed 29440 rows...
Processed 30720 rows...
Processed 32000 rows...
Processed 33280 rows...
Processed 34560 rows...
Processed 35840 rows...
Processed 37120 rows...
Processed 38400 rows...
Processed 39680 rows...
Processed 40960 rows...
Processed 42240 rows...
Processed 43520 rows...
Processed 44800 rows...
Processed 46080 rows...
Processed 47360 rows...
Processed 48640 rows...
Processed 49920 rows...
Processed 

In [27]:
device = "cpu"

# --- CONFIGURATION ---
THRESHOLD = 0.32
ANCHOR_TEXT = "Support for Ukraine against Russian aggression, providing weapons, military aid, humanitarian help, victory of Ukraine"
# ---------------------

# 2. Encode Anchor Text
# We do this first to have the vector ready
anchor_embedding = model.encode(ANCHOR_TEXT, convert_to_tensor=True, device=device)

# 3. Load Data directly into a Pandas DataFrame
print("Loading data from DuckDB into Pandas...")
with duckdb.connect(str(DB_PATH), read_only=True) as conn:
    # .df() efficiently converts the query result to a DataFrame
    df = conn.execute(
        """
        SELECT id, title, selftext, embedding
        FROM submissions_filtered
        WHERE embedding IS NOT NULL
        """
    ).df()

print(f"Loaded {len(df)} rows.")

# 4. Process Embeddings
# The embeddings are stored as JSON strings ("[-0.1, 0.5, ...]").
# We need to parse them back into Python lists/arrays.
print("Parsing JSON embeddings...")
# Using apply with json.loads is the standard way to convert stringified lists back to objects
df['embedding_vec'] = df['embedding'].apply(json.loads)

# 5. Calculate Similarity
print("Calculating cosine similarity...")

# Convert the column of lists into a single 2D Numpy array (Matrix)
# This is much faster for batch calculations than iterating row by row
corpus_embeddings = np.stack(df['embedding_vec'].values)
corpus_embeddings = torch.tensor(corpus_embeddings, device=device, dtype=torch.float32)

# Calculate scores against the anchor
cosine_scores = util.cos_sim(anchor_embedding, corpus_embeddings)[0]

# Add scores back to the DataFrame
df['score'] = cosine_scores.cpu().numpy()

# 6. Filter and Sort
pro_ukrainian_df = df[df['score'] >= THRESHOLD].copy()
pro_ukrainian_df = pro_ukrainian_df.sort_values(by='score', ascending=False)


Loading data from DuckDB into Pandas...
Loaded 98077 rows.
Parsing JSON embeddings...
Calculating cosine similarity...


In [28]:
pro_ukrainian_df[['score', 'title', "id"]]

,score,title,id
33914,0.837255,Unterstützung für die Ukraine im internationalen Vergleich – neue Entwicklungen,1lg5pzd
75452,0.823544,Support for Ukraine by US and Europe,1ms8ww1
69291,0.795644,Ukrainian Support for War Effort Collapses,1moq75g
68153,0.795644,Ukrainian Support for War Effort Collapses,1mo4t7y
61724,0.795644,Ukrainian Support for War Effort Collapses,1mkbodr
...,...,...,...
46313,0.320060,"Ukraine’s new top prosecutor known for high-profile cases, seen as Zelensky loyalist",1lm2e88
47674,0.320039,Vatniks: The Baltics are Bane of our existence,1lmvrco
65844,0.320028,Moscow strikes kill six in Ukraine; refineries hit in Russia,1mmrf78
7396,0.320020,"After trump spoke with Putin today, Trump said: ""President Putin did say, and very strongly, tha...",1l3oc95


### Addnitional filtering with more

In [36]:
from transformers import pipeline

CANDIDATE_LABELS = ["Pro-Ukrainian stance", "Pro-Russian stance", "Neutral news or unrelated"]
ZSC_THRESHOLD = 0.6

zsc_device = "mps" if torch.mps.is_available() else "cpu"
classifier = pipeline(
    "zero-shot-classification",
    model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli",
    device=zsc_device
)


Device set to use mps


In [43]:
pro_ukrainian_df['full_text_zsc'] = (
    pro_ukrainian_df['title'].fillna('') + " " + pro_ukrainian_df['selftext'].fillna('')
).str.strip().str.slice(0, 1000)

predicted_labels = []
confidence_scores = []

texts_to_classify = pro_ukrainian_df['full_text_zsc'].tolist()

pipeline_iterator = classifier(
    texts_to_classify,
    CANDIDATE_LABELS,
    multi_label=False,
    batch_size=16
)

print(f"Starting classification of {len(texts_to_classify)} texts...")

for res in tqdm(pipeline_iterator, total=len(texts_to_classify)):
    predicted_labels.append(res['labels'][0])
    confidence_scores.append(res['scores'][0])


Starting classification of 14988 texts...


100%|██████████| 14988/14988 [00:00<00:00, 182090.07it/s]


In [49]:
pro_ukrainian_df['zsc_label'] = predicted_labels
pro_ukrainian_df['zsc_score'] = confidence_scores

final_verified_df = pro_ukrainian_df[
    (pro_ukrainian_df['zsc_label'] == "Pro-Ukrainian stance") &
    (pro_ukrainian_df['zsc_score'] >= 0.3)
].copy()

final_verified_df

,id,title,selftext,embedding,embedding_vec,score,full_text_zsc,zsc_label,zsc_score
33914,1lg5pzd,Unterstützung für die Ukraine im internationalen Vergleich – neue Entwicklungen,None,"[-0.15585365891456604, 0.1458481252193451, -0.28222495317459106, -0.02534978836774826, 0.4212956...","[-0.15585365891456604, 0.1458481252193451, -0.28222495317459106, -0.02534978836774826, 0.4212956...",0.837255,Unterstützung für die Ukraine im internationalen Vergleich – neue Entwicklungen,Pro-Ukrainian stance,0.748331
75452,1ms8ww1,Support for Ukraine by US and Europe,None,"[-0.2187863141298294, 0.0850972980260849, -0.44099149107933044, -0.163397878408432, 0.5220111012...","[-0.2187863141298294, 0.0850972980260849, -0.44099149107933044, -0.163397878408432, 0.5220111012...",0.823544,Support for Ukraine by US and Europe,Pro-Ukrainian stance,0.864739
69291,1moq75g,Ukrainian Support for War Effort Collapses,None,"[-0.2132752388715744, 0.12042645364999771, -0.049056608229875565, 0.03084695339202881, 0.1952361...","[-0.2132752388715744, 0.12042645364999771, -0.049056608229875565, 0.03084695339202881, 0.1952361...",0.795644,Ukrainian Support for War Effort Collapses,Pro-Ukrainian stance,0.482695
68153,1mo4t7y,Ukrainian Support for War Effort Collapses,None,"[-0.2132752388715744, 0.1204264685511589, -0.049056608229875565, 0.030846958979964256, 0.1952362...","[-0.2132752388715744, 0.1204264685511589, -0.049056608229875565, 0.030846958979964256, 0.1952362...",0.795644,Ukrainian Support for War Effort Collapses,Pro-Ukrainian stance,0.482695
61724,1mkbodr,Ukrainian Support for War Effort Collapses,None,"[-0.2132752388715744, 0.1204264685511589, -0.049056608229875565, 0.030846958979964256, 0.1952362...","[-0.2132752388715744, 0.1204264685511589, -0.049056608229875565, 0.030846958979964256, 0.1952362...",0.795644,Ukrainian Support for War Effort Collapses,Pro-Ukrainian stance,0.482695
...,...,...,...,...,...,...,...,...,...
74865,1mrvjif,"Trump says he and Putin ‘largely agreed’ on land swaps, security guarantees for Ukraine",None,"[0.13283121585845947, -0.1483430713415146, -0.08809955418109894, -0.03930467739701271, -0.249566...","[0.13283121585845947, -0.1483430713415146, -0.08809955418109894, -0.03930467739701271, -0.249566...",0.320375,"Trump says he and Putin ‘largely agreed’ on land swaps, security guarantees for Ukraine",Pro-Ukrainian stance,0.488273
44706,1llaaft,"UK-style deal is best EU will get from Trump, Lithuanian president warns",None,"[0.1616336852312088, 0.011064761318266392, -0.197923943400383, -0.24521927535533905, 0.021407378...","[0.1616336852312088, 0.011064761318266392, -0.197923943400383, -0.24521927535533905, 0.021407378...",0.320375,"UK-style deal is best EU will get from Trump, Lithuanian president warns",Pro-Ukrainian stance,0.587055
44571,1ll84f9,"UK-style deal is best EU will get from Trump, Lithuanian president warns",None,"[0.1616336852312088, 0.011064761318266392, -0.197923943400383, -0.24521927535533905, 0.021407378...","[0.1616336852312088, 0.011064761318266392, -0.197923943400383, -0.24521927535533905, 0.021407378...",0.320375,"UK-style deal is best EU will get from Trump, Lithuanian president warns",Pro-Ukrainian stance,0.587055
15068,1l7cfw3,Perussuomalaisten kansanedustajat vaativat Elokapinan kieltämistä,None,"[-0.015614575706422329, 0.08423486351966858, -0.030692974105477333, 0.09347790479660034, 0.20822...","[-0.015614575706422329, 0.08423486351966858, -0.030692974105477333, 0.09347790479660034, 0.20822...",0.320207,Perussuomalaisten kansanedustajat vaativat Elokapinan kieltämistä,Pro-Ukrainian stance,0.372140


In [50]:

with duckdb.connect(str(DB_PATH)) as conn:
    conn.register("final_verified_df_view", final_verified_df)
    conn.execute("DROP TABLE IF EXISTS final_verified")
    conn.execute("CREATE TABLE final_sub AS SELECT * FROM final_verified_df_view")

    conn.execute("DROP TABLE IF EXISTS submissions_final_verified")
    conn.execute(
        """
        CREATE TABLE submissions_final_verified AS
        SELECT s.*
        FROM submissions s
        INNER JOIN final_verified f ON f.id = s.id
        """
    )

    count = conn.execute("SELECT COUNT(*) FROM submissions_final_verified").fetchone()[0]
    print(f"Rows written to submissions_final_verified: {count}")


Rows written to submissions_final_verified: 3702


In [ ]:
%%sql
select * from submissions_final_verified
